# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import folium
# Import API key
from config import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,9.62,91,96,1.88,NZ,1713243952
1,1,buckie,57.6757,-2.9624,6.14,85,96,10.21,GB,1713243952
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,3.33,94,75,4.00,RU,1713243816
3,3,ushuaia,-54.8000,-68.3000,7.60,91,66,1.34,AR,1713243800
4,4,mitsamiouli,-11.3847,43.2844,28.29,76,100,2.42,KM,1713243953


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map. Imported the folium library to create a map object, and specified the location of the map by using the Lat and Lng mean averages.
#set the zoom level of the map to 4.
map_city = folium.Map(location=[city_data_df['Lat'].mean(), city_data_df['Lng'].mean()], zoom_start=2)

# Each city has a few more points added, with the size indicating the humidity.
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] * 0.1,  
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=row['City'] + ", Humidity: " + str(row['Humidity'])
    ).add_to(map_city)

# Display the map
map_city

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_humidity = 50
ideal_cities_df = city_data_df[city_data_df['Humidity'] < ideal_humidity]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,waddan,29.1614,16.1390,26.00,21,59,6.43,LY,1713243954
7,7,bethel,41.3712,-73.4140,13.17,46,20,0.45,US,1713243954
24,24,argostolion,38.1811,20.4890,19.30,39,100,3.83,GR,1713243962
50,50,sao vicente,-23.9631,-46.3919,26.82,26,95,0.74,BR,1713243973
52,52,tekkali,18.6167,84.2333,36.79,36,0,5.38,IN,1713243974
...,...,...,...,...,...,...,...,...,...,...
539,539,hovd,48.0056,91.6419,14.51,27,48,1.40,MN,1713244174
540,540,kyren,51.6828,102.1408,16.08,31,59,0.89,RU,1713244174
541,541,brak,27.5496,14.2714,20.98,32,41,5.74,LY,1713244175
547,547,gazojak,41.1875,61.4036,22.78,25,61,0.97,TM,1713244178


### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:

# Define the search radius
categories = "accommodation.hotel"
radius = 1000

# Base URL for Geoapify API
base_url = "https://api.geoapify.com/v2/places"

# API parameters
params = {
    "categories": categories,
    "apiKey":geoapify_key    
}

# Start the hotel search
print("Starting hotel search")

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    # Update the parameters with the current city's latitude and longitude
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Make the API request and handle potential errors
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
    except requests.RequestException as e:
        # Log any request-related errors
        print(f"Error with request for index {index}: {e}")
        hotel_df.loc[index, "Hotel Name"] = "Request error"
        continue

    # Convert the API response to JSON
    data = response.json()

    # Extract the hotel name or set a default message if no hotel is found
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        # Handle cases where no hotel is found
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results for each city
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the updated DataFrame
hotel_df


Starting hotel search
papatowai - nearest hotel: No hotel found
buckie - nearest hotel: The Brig and Barrel Hotel
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
mitsamiouli - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
waddan - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
akureyri - nearest hotel: Hótel Norðurland
iqaluit - nearest hotel: Frobisher Inn
port-aux-francais - nearest hotel: Keravel
isafjordur - nearest hotel: Hótel Horn
puerto natales - nearest hotel: Dorotea Patagonia Hostel
mundri - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
vorkuta - nearest hotel: No hotel found
aras-asan - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
isiro - nearest hotel: Hôtel Magbetu
ilulissat - nearest hot

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,papatowai,NZ,-46.5619,169.4708,91,
1,buckie,GB,57.6757,-2.9624,85,
2,petropavlovsk-kamchatsky,RU,53.0452,158.6483,94,
3,ushuaia,AR,-54.8000,-68.3000,91,
4,mitsamiouli,KM,-11.3847,43.2844,76,
...,...,...,...,...,...,...
545,tres lagoas,BR,-20.7511,-51.6783,83,
546,hengchun,TW,22.0042,120.7439,74,
547,gazojak,TM,41.1875,61.4036,25,
548,xilin hot,CN,43.9667,116.0333,16,


###  Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

params = {
    'apiKey': geoapify_key,
    "categories": categories,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
   
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

In [ ]:
if response.status_code != 200:
    print("Error in API response:", response.status_code, response.text)
else:
    data = response.json()

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_city = folium.Map(location=[hotel_df['Lat'].mean(), hotel_df['Lng'].mean()], zoom_start=4)
# Add points for each city with size representing humidity
for index, row in hotel_df.iterrows():
# Define the hover message with additional information (hotel name and country)
    hover_message = f"City: {row['City']}, Country: {row['Country']}, Hotel: {row['Hotel Name']}"
    
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=5,  # Adjusting radius for visualization
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        popup=hover_message
    ).add_to(map_city)

# Display the map
map_city